In [1]:
import spacy
import torch
from torchtext import data    
from libs import sql_tokenizer
import libs
import numpy as np
import pandas as pd

In [2]:
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [3]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [4]:
fields = [(None, None), ('text',TEXT),('label', LABEL)]

In [5]:
training_data=data.TabularDataset(path = 'csv_files/xss_safe_sql.csv',format = 'csv',fields = fields,skip_header = True)

#print preprocessed text
print(vars(training_data.examples[0]))
for i,dt in enumerate(training_data):
    if(len(dt.text) <= 0):
        training_data[i].text = "<blank>"
        print(i,training_data[i].text,training_data[i].label)

{'text': ['LT', 'script', 'GT', 'alert', 'LPRN', 'INT', 'RPRN', 'SMCLN', 'LT', 'SLSH', 'script', 'GT'], 'label': 'xss'}
19 <blank> xss
20 <blank> xss
22 <blank> xss
24 <blank> xss
26 <blank> xss
28 <blank> xss
30 <blank> xss
32 <blank> xss
34 <blank> xss
36 <blank> xss
38 <blank> xss
40 <blank> xss
42 <blank> xss
44 <blank> xss
46 <blank> xss
48 <blank> xss
50 <blank> xss
52 <blank> xss
55 <blank> xss
57 <blank> xss
61 <blank> xss
63 <blank> xss
65 <blank> xss
67 <blank> xss
69 <blank> xss
71 <blank> xss
73 <blank> xss
75 <blank> xss
77 <blank> xss
79 <blank> xss
81 <blank> xss
83 <blank> xss
85 <blank> xss
87 <blank> xss
89 <blank> xss
91 <blank> xss
93 <blank> xss
95 <blank> xss
97 <blank> xss
99 <blank> xss
101 <blank> xss
103 <blank> xss
105 <blank> xss
107 <blank> xss
109 <blank> xss
111 <blank> xss
113 <blank> xss
115 <blank> xss
117 <blank> xss
119 <blank> xss
121 <blank> xss
123 <blank> xss
125 <blank> xss
127 <blank> xss
129 <blank> xss
131 <blank> xss
133 <blank> xss
135 <bla

In [6]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)
print("Size of TEXT vocabulary:",len(TEXT.vocab))
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
#print(TEXT.vocab.freqs.most_common(100))  
print(TEXT.vocab.stoi)   

Size of TEXT vocabulary: 1726
Size of LABEL vocabulary: 4


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000001F89A6FA860>>, {'<unk>': 0, '<pad>': 1, 'DOT': 2, 'the': 3, 'INT': 4, 'RPRN': 5, 'CMMA': 6, 'A': 7, 'SMCLN': 8, 'AND': 9, 'BITAND': 10, 'LPRN': 11, 'SQUT': 12, 'OF': 13, 'NULL': 14, 'TO': 15, 'SLSH': 16, 'IS': 17, 'i': 18, 'br': 19, 'it': 20, 'PRCNT': 21, 'EQ': 22, 'IN': 23, 'DQUT': 24, 'that': 25, 'this': 26, 'MINUS': 27, 'HASH': 28, 'GT': 29, 'LT': 30, 's': 31, 'movie': 32, 'CLN': 33, 'was': 34, 'BSLSH': 35, 'AS': 36, 'but': 37, 'WITH': 38, 'film': 39, 'ON': 40, 'FOR': 41, 'script': 42, 'T': 43, 'NOT': 44, 'FROM': 45, 'you': 46, 'alert': 47, 'STAR': 48, 'OR': 49, 'IF': 50, 'ARE': 51, 'ALL': 52, 'be': 53, 'tab': 54, 'they': 55, 'have': 56, 'one': 57, 'gt': 58, 'lt': 59, 'an': 60, 'his': 61, 'xss': 62, 'he': 63, 'SELECT': 64, 'who': 65, 'benchmark': 66, 'md5': 67, 'AT': 68, 'so': 69, 'PLUS': 70, 'BY': 71, 'LIKE': 72, 'story': 73, 'src': 74, 'OUT': 75, 'just': 76, 'good': 77, 'titanic': 78, 'ca

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [8]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
#        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
#packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs= dense_outputs#self.act(dense_outputs)
        
        return outputs

In [9]:

#LABEL.vocab.stoi.pop("label")
print(LABEL.vocab.stoi)

defaultdict(None, {'sql': 0, 'xss': 1, 'safe': 2, 'label': 3})


In [10]:
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = len(LABEL.vocab.stoi)
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [11]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(1726, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=4, bias=True)
)
The model has 232,252 trainable parameters
torch.Size([1726, 100])


In [12]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_pred = torch.round(preds)
    _,pred_label = torch.max(rounded_pred, dim = 1)
    correct = (pred_label == y).float()
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [13]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        y_tensor = torch.tensor(batch.label, dtype=torch.long, device=device)
        loss = criterion(predictions, y_tensor)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [14]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            y_tensor = torch.tensor(batch.label, dtype=torch.long, device=device)
            loss = criterion(predictions, y_tensor)      
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
N_EPOCHS = 7
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


	Train Loss: 1.334 | Train Acc: 0.10%
	 Val. Loss: 1.238 |  Val. Acc: 0.46%
	Train Loss: 1.144 | Train Acc: 26.17%
	 Val. Loss: 1.030 |  Val. Acc: 74.11%
	Train Loss: 0.860 | Train Acc: 60.91%
	 Val. Loss: 0.677 |  Val. Acc: 79.35%
	Train Loss: 0.460 | Train Acc: 88.57%
	 Val. Loss: 0.312 |  Val. Acc: 93.74%
	Train Loss: 0.225 | Train Acc: 95.90%
	 Val. Loss: 0.190 |  Val. Acc: 95.06%
	Train Loss: 0.143 | Train Acc: 96.39%
	 Val. Loss: 0.129 |  Val. Acc: 95.51%
	Train Loss: 0.111 | Train Acc: 96.19%
	 Val. Loss: 0.152 |  Val. Acc: 94.85%


In [16]:
import time
import os
# lets save the model
def save_model():
    models_path = "saved_weights"
    md_val_acc = "%.2f"%(valid_acc*100)
    model_name = "Acc "+md_val_acc+".pt"
    full_path = os.path.join(models_path, model_name)
    torch.save(model.state_dict(),full_path)
    print("SAVED\n",model_name)

In [17]:
save_model()

SAVED
 Acc 94.85.pt


In [23]:
new_list = LABEL.vocab.stoi.pop("label")
new_list

3

In [18]:
import spacy

nlp = spacy.load('en')

def predict(model,sentence):
    pred_2_lbl = {1:'xss',0:'sql',2:"s"}
    tokenized = [tok.text for tok in nlp.tokenizer(sql_tokenizer(sentence))] # tokenize the sentence
    print(tokenized)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized] # convert to integer sequence
    print(indexed)
    length = [len(indexed)] #compute no. of words
    tensor = torch.LongTensor(indexed).to(device) # convert to tensor
    tensor = tensor.unsqueeze(1).T
    length_tensor = torch.LongTensor(length)
    prediction = model(tensor,length_tensor)
    pred_lbl = np.argmax(prediction.detach().numpy())
    print('\n')
    print('predicted threat type:',pred_2_lbl[pred_lbl])
    return prediction

In [19]:
pred = predict(model,""" SELECT * FROM items
WHERE owner = 'wiley'
AND itemname = 'name' OR 'a'='a'; """)

[' ', 'SELECT', 'STAR', 'FROM', 'items', '\n', 'where', 'OWNER', 'EQ', 'SQUT', 'wiley', 'SQUT', 'AND', 'itemname', 'EQ', 'SQUT', 'NAME', 'SQUT', 'OR', 'SQUT', 'A', 'SQUT', 'EQ', 'SQUT', 'A', 'SQUT', 'SMCLN']
[0, 64, 48, 45, 0, 0, 0, 0, 22, 12, 0, 12, 9, 0, 22, 12, 379, 12, 49, 12, 7, 12, 22, 12, 7, 12, 8]


predicted threat type: sql


In [20]:
np.argmax(pred.detach().numpy())

0

In [21]:
LABEL.vocab.stoi.items()

dict_items([('sql', 0), ('xss', 1), ('safe', 2), ('label', 3)])

# Safe Text

In [22]:
predict(model,'im good')

['i', 'm', 'good']
[18, 837, 77]




KeyError: 2